# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [11]:
# Import libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
from nltk.probability import ConditionalFreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
# your code here -- I had to clean it by myself because I was having problems importing the dataset
tweets = pd.read_excel("dataset.xlsx").sample(20000)
tweets

,target,id,date,flag,user,text
862503,4,1676911040,Fri May 01 23:40:45 PDT 2009,NO_QUERY,annabellab8,is DONE 4 the night... Goin to bed early to wa...
735818,0,2264874314,Sun Jun 21 05:11:10 PDT 2009,NO_QUERY,HloveN,Anh muá»‘n nghe giá»?ng nÃ³i cá»§a thiÃªn tháº...
337708,0,2014270812,Wed Jun 03 00:59:21 PDT 2009,NO_QUERY,Blears,trust me to be off college but have to do the ...
186842,0,1968432575,Fri May 29 21:25:35 PDT 2009,NO_QUERY,bunz_bunz,Today is my birthday. I am now 29 going on 30....
619015,0,2227761384,Thu Jun 18 13:36:03 PDT 2009,NO_QUERY,samuraikitten,I forgot tonight was Third eye blind. Never di...
...,...,...,...,...,...,...
121366,0,1833494768,Sun May 17 23:28:08 PDT 2009,NO_QUERY,atulkedia,Struggling with two summer projects at the sam...
554007,0,2203752365,Wed Jun 17 00:27:11 PDT 2009,NO_QUERY,DeannaPHX,happy birthday to me.
298209,0,1997615871,Mon Jun 01 17:09:34 PDT 2009,NO_QUERY,KatherineJessen,omg i am sooo sore this is not getting better
912956,4,1752446215,Sat May 09 21:56:54 PDT 2009,NO_QUERY,listenupjulia,@JanisJ1212 haven't laughed that much in awhile


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [14]:
# 1 - Text cleaning
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string. 
    ## stackoverflow: re.sub(r'http\S+', '', stringliteral)

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    ## lets save a cleaned phrase using the re.substitute function -- stackoverflow: (str.replace(/[^A-Za-z ]/g, "")
    clean = re.sub(r'http\S+', '', s)
    return re.sub('[^A-Za-z ]+', ' ', clean).lower().strip()

In [15]:
# 2 - Tokenization
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    return nltk.word_tokenize(s)

In [16]:
# 3 - Stem and Lemmatize
def stem_and_lemmatize(l):
    """
    Perform stemming and lemmatization on a list of words.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after being stemmed and lemmatized.
    """
    ## first define porter stemmer and lemmatizer
    
    ps = nltk.PorterStemmer()
    lemmatizer = nltk.WordNetLemmatizer()
    list_words = []
    
    for w in l:
        s = ps.stem(w)
        s = lemmatizer.lemmatize(s)
        list_words += [s]   # es lo mismo que append
    return list_words

In [17]:
# 4 - Stop words removal
def remove_stopwords(l):
    """
    Remove English stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    stop_words = stopwords.words('english')
    return[w for w in l if w not in stop_words]

In [18]:
# Lets apply these functions
# Multiple can be applied at the same time, StackOverflow eg: df.authors.apply(eval).apply(len).sum() 
tweets['text'] = tweets['text'].apply(str)  # change to string for avoiding errors

In [19]:
tweets['text_processed'] = tweets['text'].apply(clean_up).apply(tokenize).apply(stem_and_lemmatize).apply(remove_stopwords)
tweets

,target,id,date,flag,user,text,text_processed
862503,4,1676911040,Fri May 01 23:40:45 PDT 2009,NO_QUERY,annabellab8,is DONE 4 the night... Goin to bed early to wa...,"[done, night, goin, bed, earli, wake, time, sh..."
735818,0,2264874314,Sun Jun 21 05:11:10 PDT 2009,NO_QUERY,HloveN,Anh muá»‘n nghe giá»?ng nÃ³i cá»§a thiÃªn tháº...,"[anh, mu, n, nghe, gi, ng, n, c, thi, n, th, n..."
337708,0,2014270812,Wed Jun 03 00:59:21 PDT 2009,NO_QUERY,Blears,trust me to be off college but have to do the ...,"[trust, colleg, one, thing, hate, babysit]"
186842,0,1968432575,Fri May 29 21:25:35 PDT 2009,NO_QUERY,bunz_bunz,Today is my birthday. I am now 29 going on 30....,"[today, birthday, go, man, sound, old]"
619015,0,2227761384,Thu Jun 18 13:36:03 PDT 2009,NO_QUERY,samuraikitten,I forgot tonight was Third eye blind. Never di...,"[forgot, tonight, wa, third, eye, blind, never..."
...,...,...,...,...,...,...,...
121366,0,1833494768,Sun May 17 23:28:08 PDT 2009,NO_QUERY,atulkedia,Struggling with two summer projects at the sam...,"[struggl, two, summer, project, time]"
554007,0,2203752365,Wed Jun 17 00:27:11 PDT 2009,NO_QUERY,DeannaPHX,happy birthday to me.,"[happi, birthday]"
298209,0,1997615871,Mon Jun 01 17:09:34 PDT 2009,NO_QUERY,KatherineJessen,omg i am sooo sore this is not getting better,"[omg, sooo, sore, thi, get, better]"
912956,4,1752446215,Sat May 09 21:56:54 PDT 2009,NO_QUERY,listenupjulia,@JanisJ1212 haven't laughed that much in awhile,"[janisj, laugh, much, awhil]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
# your code here
bag = nltk.FreqDist()

In [20]:
# define the function
for tweet in tweets['text_processed']:
    for word in tweet:
        bag[word] += 1

In [23]:
# create tje top words list
top_words = list(bag.keys())[:5000]
top_words

['done',
 'night',
 'goin',
 'bed',
 'earli',
 'wake',
 'time',
 'shabbat',
 'tomorrow',
 'anh',
 'mu',
 'n',
 'nghe',
 'gi',
 'ng',
 'c',
 'thi',
 'th',
 'trust',
 'colleg',
 'one',
 'thing',
 'hate',
 'babysit',
 'today',
 'birthday',
 'go',
 'man',
 'sound',
 'old',
 'forgot',
 'tonight',
 'wa',
 'third',
 'eye',
 'blind',
 'never',
 'find',
 'anyon',
 'everyon',
 'follow',
 'bff',
 'coolest',
 'ever',
 'meet',
 'gt',
 'melodatrichboi',
 'blairbonaldi',
 'let',
 'help',
 'start',
 'eat',
 'chocol',
 'miss',
 'monkey',
 'pretend',
 'pillow',
 'hi',
 'chest',
 'lekaslah',
 'berlabuh',
 'jare',
 'mckendal',
 'tweep',
 'crazi',
 'pretzel',
 'much',
 'crap',
 'probabl',
 'ani',
 'haha',
 'fight',
 'layout',
 'wxhaskel',
 'make',
 'textctrl',
 'fill',
 'entir',
 'vertic',
 'space',
 'resiz',
 'window',
 'stuck',
 'lite',
 'almost',
 'min',
 'late',
 'class',
 'omg',
 'right',
 'smh',
 'wen',
 'ha',
 'work',
 'next',
 'morn',
 'amp',
 'alon',
 'whi',
 'head',
 'home',
 'certainli',
 'seem'

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [30]:
# your code here
def find_features(document):
    words = set(document)
    features = {}
    for w in top_words:
        features[w] = (w in words)

    return features

In [31]:
feature_sets = [(find_features(tweet), target) for (tweet, target) in list(zip(tweets['text_processed'], tweets['target']))]
print(len(feature_sets))

20000


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [32]:
# your code here
training_set = feature_sets[:9000] 
testing_set = feature_sets[9000:]
classifier = nltk.NaiveBayesClassifier.train(training_set)

### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [34]:
# your code here
print(nltk.classify.accuracy(classifier, testing_set))
classifier.show_most_informative_features(5) #yaaaay is over 0.7!

0.7853
Most Informative Features
                  welcom = True                4 : 0      =     19.2 : 1.0
                     men = True                4 : 0      =     15.6 : 1.0
                     sad = True                0 : 4      =     15.0 : 1.0
                 appreci = True                4 : 0      =     13.6 : 1.0
                   thanx = True                4 : 0      =     13.6 : 1.0


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here